# Loan Data Preprocessing Pipeline

This notebook performs comprehensive data preprocessing on loan data, including handling missing values, removing irrelevant features, and preparing the dataset for modeling.

## 1. Import Libraries and Load Data

In [52]:
import pandas as pd
import numpy as np

def load_data(file_path):
    """Load CSV data into a DataFrame."""
    df = pd.read_csv(file_path, low_memory=False)
    
    print(f"Data loaded. Shape: {df.shape}")
    return df

In [41]:
# Load your data
df = load_data('../data/raw/sample.csv')

Data loaded. Shape: (100000, 152)


## 2. Drop Completely Empty Columns

In [42]:
def drop_empty_columns(df):
    """Drop columns that are completely empty."""
    empty_columns = df.columns[df.isna().all()]
    df.drop(empty_columns, axis=1, inplace=True)
    print(f"Dropped {len(empty_columns)} empty columns")
    return df

df = drop_empty_columns(df)

Dropped 1 empty columns


## 3. Clean Loan Status

Simplify loan_status to binary classification: Fully Paid or Default

In [43]:
def clean_loan_status(df):
    """Simplify loan_status to Fully Paid or Default."""
    df['loan_status'] = df['loan_status'].replace({'Charged Off': 'Default'})
    df = df[df['loan_status'].isin(['Fully Paid', 'Default'])]
    print("\nLoan status distribution:")
    print(df['loan_status'].value_counts())
    return df

df = clean_loan_status(df)


Loan status distribution:
loan_status
Fully Paid    47308
Default       12089
Name: count, dtype: int64


## 4. Drop Columns with High Missing Values (>60%)

In [44]:
def drop_high_missing(df, threshold=60):
    """Drop columns with more than threshold% missing values."""
    percent_missing = df.isnull().sum() * 100 / len(df)
    cols_to_drop = percent_missing[percent_missing >= threshold].index
    df.drop(cols_to_drop, axis=1, inplace=True)
    print(f"\nDropped {len(cols_to_drop)} columns with >={threshold}% missing values")
    return df

df = drop_high_missing(df, threshold=60)


Dropped 56 columns with >=60% missing values


## 5. Analyze and Drop High Cardinality Categorical Columns

**Columns removed due to high unique values and being categorical:**
- `id`: Unique identifier (no predictive value)
- `url`: URL to loan page (no predictive value)
- `policy_code`: Single unique value
- `title`: High cardinality text field
- `zip_code`: High cardinality location data
- `pymnt_plan`: Very low variance
- `emp_title`: Extremely high cardinality (job titles)
- `addr_state`: Can be dropped if geographic info not needed

In [45]:
def analyze_high_cardinality(df):
    """Analyze categorical columns for high cardinality."""
    cols_to_check = ['id', 'url', 'policy_code', 'title', 'zip_code', 
                     'pymnt_plan', 'emp_title', 'addr_state']
    
    print("="*70)
    print("HIGH CARDINALITY CATEGORICAL ANALYSIS")
    print("="*70)
    
    for col in cols_to_check:
        if col in df.columns:
            n_unique = df[col].nunique()
            total_rows = len(df)
            pct_unique = (n_unique / total_rows) * 100
            
            print(f"\n{col}:")
            print(f"  - Unique values: {n_unique:,} ({pct_unique:.2f}% of total rows)")
            print(f"  - Data type: {df[col].dtype}")
            
            if n_unique <= 10:
                print(f"  - Value counts:\n{df[col].value_counts()}")

# Run analysis to see why these columns should be dropped
analyze_high_cardinality(df)

HIGH CARDINALITY CATEGORICAL ANALYSIS

id:
  - Unique values: 59,397 (100.00% of total rows)
  - Data type: object

url:
  - Unique values: 59,397 (100.00% of total rows)
  - Data type: object

policy_code:
  - Unique values: 1 (0.00% of total rows)
  - Data type: float64
  - Value counts:
policy_code
1.0    59397
Name: count, dtype: int64

title:
  - Unique values: 4,134 (6.96% of total rows)
  - Data type: object

zip_code:
  - Unique values: 869 (1.46% of total rows)
  - Data type: object

pymnt_plan:
  - Unique values: 1 (0.00% of total rows)
  - Data type: object
  - Value counts:
pymnt_plan
n    59397
Name: count, dtype: int64

emp_title:
  - Unique values: 28,449 (47.90% of total rows)
  - Data type: object

addr_state:
  - Unique values: 50 (0.08% of total rows)
  - Data type: object


In [46]:
def drop_irrelevant_columns(df):
    """Drop ID, URL, and other irrelevant/redundant columns."""
    cols_to_drop = ['id', 'url', 'policy_code', 'title', 'zip_code', 
                    'pymnt_plan', 'emp_title', 'addr_state']
    
    existing_cols = [col for col in cols_to_drop if col in df.columns]
    df.drop(columns=existing_cols, inplace=True)
    print(f"\nDropped {len(existing_cols)} irrelevant/high cardinality columns")
    return df

df = drop_irrelevant_columns(df)


Dropped 8 irrelevant/high cardinality columns


## 6. Analyze and Drop Highly Correlated Columns

**Columns removed due to high correlation:**
- `funded_amnt`: Highly correlated with loan_amnt
- `funded_amnt_inv`: Highly correlated with loan_amnt
- `out_prncp_inv`: Highly correlated with other principal columns

**Columns removed due to no variance:**
- `hardship_flag`: Only one unique value

In [47]:
def analyze_correlations_and_variance(df):
    """Analyze specific columns for correlation and variance issues."""
    print("="*70)
    print("CORRELATION AND VARIANCE ANALYSIS")
    print("="*70)
    
    # Check hardship_flag for unique values
    if 'hardship_flag' in df.columns:
        print("\nhardship_flag:")
        print(f"  - Unique values: {df['hardship_flag'].nunique()}")
        print(f"  - Value counts:\n{df['hardship_flag'].value_counts()}")
    
    # Check correlation between amount columns
    amount_cols = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'out_prncp_inv']
    existing_amount_cols = [col for col in amount_cols if col in df.columns]
    
    if len(existing_amount_cols) > 1:
        print("\n\nCorrelation matrix for amount columns:")
        corr_matrix = df[existing_amount_cols].corr()
        print(corr_matrix.round(3))

# Run analysis to see correlations
analyze_correlations_and_variance(df)

CORRELATION AND VARIANCE ANALYSIS

hardship_flag:
  - Unique values: 1
  - Value counts:
hardship_flag
N    59397
Name: count, dtype: int64


Correlation matrix for amount columns:
                 loan_amnt  funded_amnt  funded_amnt_inv  out_prncp_inv
loan_amnt            1.000        1.000            0.999          0.004
funded_amnt          1.000        1.000            0.999          0.004
funded_amnt_inv      0.999        0.999            1.000          0.004
out_prncp_inv        0.004        0.004            0.004          1.000


In [48]:
def drop_highly_correlated(df, threshold=0.9):
    """Drop highly correlated columns, keeping the one with highest variance."""
    numeric_df = df.select_dtypes(include='number')
    corr_matrix = numeric_df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = set()

    for col in upper.columns:
        high_corr = upper[col][upper[col] > threshold].index.tolist()
        for correlated_col in high_corr:
            if correlated_col not in to_drop and col not in to_drop:
                if numeric_df[col].var() >= numeric_df[correlated_col].var():
                    to_drop.add(correlated_col)
                else:
                    to_drop.add(col)

    df.drop(columns=list(to_drop), inplace=True)
    print(f"\nDropped {len(to_drop)} highly correlated columns (threshold={threshold})")
    if to_drop:
        print(f"Columns dropped: {list(to_drop)}")
    return df

df = drop_highly_correlated(df, threshold=0.9)


Dropped 11 highly correlated columns (threshold=0.9)
Columns dropped: ['out_prncp', 'total_rec_prncp', 'num_rev_tl_bal_gt_0', 'funded_amnt', 'tot_cur_bal', 'fico_range_low', 'open_acc', 'loan_amnt', 'installment', 'collection_recovery_fee', 'total_pymnt']


## 7. Process Date Columns

Extract year and month from date columns and drop original date columns

In [49]:
def process_dates(df):
    """Extract features from date columns and drop originals."""
    # Process sub_grade (convert letter-number to numeric)
    if 'sub_grade' in df.columns:
        df['sub_grade'] = df['sub_grade'].str[1:].fillna(-1).astype(int)
    
    # Date columns to convert
    date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
    processed = 0
    
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], format='%b-%Y', errors='coerce')
            df[col + '_year'] = df[col].dt.year
            df[col + '_month'] = df[col].dt.month
            df.drop(columns=[col], inplace=True)
            processed += 1
    
    print(f"\nProcessed {processed} date columns (extracted year/month features)")
    return df

df = process_dates(df)


Processed 4 date columns (extracted year/month features)


## 8. Fill Missing Values

- Numeric columns: filled with median
- Categorical columns: filled with 'missing' category

In [50]:
def fill_missing_values(df):
    """Fill missing values appropriately by data type."""
    # Fill numeric columns with median
    numeric_cols = df.select_dtypes(include='number').columns
    filled_numeric = 0
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())
            filled_numeric += 1
    
    # Fill categorical/string columns with 'missing'
    categorical_cols = ['disbursement_method', 'debt_settlement_flag',
                        'application_type', 'initial_list_status',
                        'purpose', 'loan_status', 'verification_status',
                        'home_ownership', 'emp_length', 'grade', 'term']
    
    filled_cat = 0
    for col in categorical_cols:
        if col in df.columns and df[col].isnull().sum() > 0:
            df[col] = df[col].fillna('missing')
            filled_cat += 1
    
    print(f"\nFilled missing values in {filled_numeric} numeric columns (median)")
    print(f"Filled missing values in {filled_cat} categorical columns (with 'missing')")
    return df

df = fill_missing_values(df)


Filled missing values in 43 numeric columns (median)
Filled missing values in 1 categorical columns (with 'missing')


## 9. Final Summary

In [51]:
print("="*70)
print("PREPROCESSING COMPLETE")
print("="*70)
print(f"\nFinal shape: {df.shape}")
print(f"\nRemaining columns: {df.columns.tolist()}")
print(f"\nMissing values remaining: {df.isnull().sum().sum()}")

PREPROCESSING COMPLETE

Final shape: (59397, 80)

Remaining columns: ['Unnamed: 0', 'funded_amnt_inv', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'loan_status', 'purpose', 'dti', 'delinq_2yrs', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp_inv', 'total_pymnt_inv', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'last_pymnt_amnt', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'application_type', 'acc_now_delinq', 'tot_coll_amt', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num

## Summary

**Key preprocessing steps completed:**

1. ✓ Removed empty columns
2. ✓ Simplified loan_status to binary (Fully Paid/Default)
3. ✓ Dropped columns with >60% missing values
4. ✓ Removed high cardinality categorical features (proved with analysis)
5. ✓ Removed highly correlated numeric features (proved with correlation matrix)
6. ✓ Removed zero-variance features (hardship_flag)
7. ✓ Extracted temporal features from dates
8. ✓ Filled remaining missing values

**Dataset is now ready for modeling!**